In [13]:
# !pip install requests

In [14]:
# !pip install beautifulsoup4

In [15]:
# !pip install tqdm

# 문제점

1. 암호화폐 뉴스 사이트들 대부분이 서로 크롤링 하는 경우가 많아서 이미 해당 사건으로부터 몇 십분에서 몇 시간, 며칠이 지난 후에 뉴스가 올라오는 경우가 되게 많음

예: A사이트는 B사이트를 크롤링해서 뉴스 게재 -> B사이트는 C사이트를 크롤링해서 뉴스 게재 -> C사이트는 A사이트를 크롤링해서 게재

2. 원인 불명의 문제로 인해 크롤링할 때 일부 뉴스 데이터들이 누락

# 라이브러리

In [7]:
# 파이썬 표준 라이브러리
import os
import json
import re
import random
import time
import traceback
from datetime import datetime
from functools import partial
from concurrent import futures

# 파이썬 서드파티 라이브러리
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# 함수

In [8]:
def request_html(
                url: str, headers: dict[str, str], allow_redirects: bool, timeout: int,
                 max_retry: int
                 ) -> str | None:
    """requests로 HTML 문서 정보를 불러오는 함수

    Args:
        url: URL
        headers: 식별 정보
        allow_redirects: 리다이렉트 허용 여부
        timeout: 응답 대기 허용 시간
        max_retry: HTML 문서 정보 불러오기에 실패했을 때 재시도할 최대 횟수
    
    Return:
        텍스트화한 HTML 문서 정보, str

        or 
    
        None
    """

    html = None

    for _ in range(max_retry):
        # requests로 HTML GET
        response = requests.get(url, headers=headers, allow_redirects=allow_redirects, timeout=timeout)
        # HTML 문서 정보를 불러오는 것에 성공하면 for문 중단
        if response.ok and response.status_code == requests.codes.ok:
            html = response.text
            break

        time.sleep(random.uniform(0.5, 1.25))
    
    # 응답 요청이 실패했으면 메세지 출력
    if html is None:
        print()
        print(response.reason)
        print(f'HTML 문서 정보 가져오기를 실패한 URL : {url}')
    
    return html

In [9]:
def investing(
            news_url: str, headers: dict[str, str], allow_redirects: bool, timeout: int, max_retry: int,
              p0: re.Pattern, p1: re.Pattern, p2: re.Pattern, p3: re.Pattern, p4: re.Pattern, p5: re.Pattern
              ) -> dict[str, str, None] | None:
    """뉴스 URL을 바탕으로 크롤링을 하는 함수

    Args:
        news_url_tag: 뉴스 URL
        headers: 식별 정보
        allow_redirects: 리다이렉트 허용 여부
        timeout: 응답 대기 허용 시간
        max_retry: HTML 문서 정보 불러오기에 실패했을 때 재시도할 최대 횟수
        p0: "입력: \r\n 2024- 12- 07- 오후 08:45"와 같은 텍스트에서 ' \r\n '를 기준으로 분리하는 패턴
        p1: \n가 1개 이상 연속되는 패턴
        p2: 디셉터에서 읽기 / Provided COINNESS / 이승훈 기자 123@gmail.com 등의 불필요한 텍스트 패턴
        p3: "습니다.그 결과,"와 같은 텍스트에서 마침표 다음 띄워쓰기를 보정
        p4: "필요하다.━AI로"와 같은 텍스트에서 마침표 다음 줄바꿈을 보정
        p5: 줄바꿈이 3번 이상 연속하면 줄바꿈 2번으로 보정

    Returns:
        {
            "news_title": 뉴스 제목, str
            "news_first_upload_time": 뉴스 최초 업로드 시각, str | None
            "newsfinal_upload_time": 뉴스 최종 수정 시각, str | None
            "author": 뉴스 작성자, str | None
            "news_content": 뉴스 본문, str
            "news_url": 뉴스 URL, str
            "news_website": 뉴스 웹사이트, str
            "note": 비고, str | None
        }

        or

        None
    """
    info = {} # 뉴스 데이터 정보 Dictionary

    # requests로 HTML GET
    html = request_html(url=news_url, headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry)
    # HTML 문서 정보를 불러오는 것에 실패하면 None 반환
    if html is None:
        return None
    # BeautifulSoup로 parser
    soup = BeautifulSoup(html, 'html.parser')

    # 1. 뉴스 데이터의 제목
    title = soup.find('h1', id='articleTitle')
    title = title.text.strip(' \t\n\r\f\v')

    # 2. 뉴스 데이터의 최초 업로드 시각과 최종 수정 시각
    div = soup.find_all('div', {'class': 'flex flex-row items-center'})
    span = div[1].find('span')
    span = span.text.strip(' \t\n\r\f\v')

    first_upload_time_list = p0.split(string=span)[1].split()
    y_m_d = '-'.join(times[:-1] for times in first_upload_time_list[:3])
    if first_upload_time_list[3] == '오전':
        ap = 'AM'
    else:
        ap = 'PM'

    first_upload_time = y_m_d + ' ' + ap + ' ' + first_upload_time_list[4]
    last_upload_time = None


    # 3. 뉴스 데이터의 기사 작성자
    author = None

    # 4. 뉴스 데이터의 본문
    article = soup.find('div', id='article')

    # td 태그들은 모두 제거
    # a 태그인 텍스트들 중 p2 패턴이 있는 텍스트는 아래 데이터 전처리1에서 걸러지기 쉽게 앞/뒤로 \n 추가
    # li 태그 밑의 하위 태그들을 텍스트만 남기고 모두 제거
    # em 태그의 시작에 \n\n을 추가
    for tag in article.find_all(['td', 'a', 'li', 'em']):
        match tag.name:
            case 'td':
                tag.decompose()
            case 'a':
                if p2.search(string=tag.text) is not None:
                    tag.string = f'\n{tag.text}\n'
                tag.unwrap()
            case 'li':
                text_content = ''.join(tag.stripped_strings)
                tag.clear()  # 하위 태그 모두 제거
                tag.string = f'\n\n-> {text_content}\n\n'  # 텍스트만 남김
            case 'em':
                tag.string = f'\n\n{tag.text}'
    
    all_text = article.text.strip(' \t\n\r\f\v')
    news_content = p1.split(string=all_text)
    # 뉴스 데이터 본문의 데이터 전처리1
    # 디셉터에서 읽기 / Provided COINNESS / 이승훈 기자 123@gmail.com 등의 불필요한 텍스트 제거
    while news_content:
        if  p2.search(string=news_content[-1]) is None:
            break
        del news_content[-1]
        
    # 뉴스 데이터 본문의 데이터 전처리2
    # - 하나의 텍스트로 결합하되, 사이에 줄바꿈 2개 추가
    # - 띄워쓰기 보정
    # - 줄바꿈 보정1, 2
    # - 맨 앞/뒤의 화이트스페이스(whitespace) 문자 제거
    content = '\n\n'.join(news_content)
    content = p3.sub(repl=r'\g<1>. \g<2>', string=content)
    content = p4.sub(repl=r'.\n\n━', string=content)
    content = p5.sub(repl=r'\n\n', string=content)
    content = content.strip(' \t\n\r\f\v')

    # 5. 뉴스 웹사이트 이름
    website = 'Investing'

    # 6. 뉴스 카테고리
    category = None
    
    # 7. 비고
    note = None

    info['news_title'] = title
    info['news_first_upload_time'] = first_upload_time
    info['news_last_upload_time'] = last_upload_time
    info['author'] = author
    info['news_content'] = content
    info['news_url'] = news_url
    info['news_website'] = website
    info['news_category'] = category
    info['note'] = note

    return info

In [10]:
def hankyung(
            news_url: str, headers: dict[str, str], allow_redirects: bool, timeout: int, max_retry: int,
              p0: re.Pattern
              ) -> dict[str, str, None] | None:
    """뉴스 URL을 바탕으로 크롤링을 하는 함수

    Args:
        news_url_tag: 뉴스 URL
        headers: 식별 정보
        allow_redirects: 리다이렉트 허용 여부
        timeout: 응답 대기 허용 시간
        max_retry: HTML 문서 정보 불러오기에 실패했을 때 재시도할 최대 횟수
        p0: 인도 벵갈루루=양한나 블루밍비트 기자 sheep@bloomingbit.io / (사진=연합뉴스)와같은 텍스트 패턴

    Returns:
        {
            "news_title": 뉴스 제목, str
            "news_first_upload_time": 뉴스 최초 업로드 시각, str | None
            "newsfinal_upload_time": 뉴스 최종 수정 시각, str | None
            "author": 뉴스 작성자, str | None
            "news_content": 뉴스 본문, str
            "news_url": 뉴스 URL, str
            "news_website": 뉴스 웹사이트, str
            "note": 비고, str | None
        }

        or

        None
    """
    info = {} # 뉴스 데이터 정보 Dictionary

    # requests로 HTML GET
    html = request_html(url=news_url, headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry)
    # HTML 문서 정보를 불러오는 것에 실패하면 None 반환
    if html is None:
        return None
    # BeautifulSoup로 parser
    soup = BeautifulSoup(html, 'html.parser')

    # 1. 뉴스 데이터의 제목
    title = soup.find('h1', {"class": "headline"})
    title = title.text.strip(' \t\n\r\f\v')

    # 2. 뉴스 데이터의 최초 업로드 시각과 최종 수정 시각
    upload_times = soup.find_all('span', {"class": "txt-date"})

    first_upload_time = upload_times[0].text
    first_upload_time = datetime.strptime(first_upload_time, '%Y.%m.%d %H:%M')
    first_upload_time = datetime.strftime(first_upload_time, '%Y-%m-%d %p %I:%M')
    last_upload_time = upload_times[1].text
    last_upload_time = datetime.strptime(last_upload_time, '%Y.%m.%d %H:%M')
    last_upload_time = datetime.strftime(last_upload_time, '%Y-%m-%d %p %I:%M')

    # 3. 뉴스 데이터의 기사 작성자
    author_list = soup.find_all('div', {"class": "author link subs_author_list"})
    author_list = map(lambda x: x.find("a").text, author_list)
    author = ', '.join(author_list)

    # 4. 뉴스 데이터의 본문
    articletxt = soup.find("div", id="articletxt")

    # strong, figcaption 태그들을 모두 제거
    for tag in articletxt.find_all(['strong', 'figcaption']):
        tag.decompose()

    text_list = articletxt.get_text(separator="\n", strip=True).split('\n')
    # "사진=", "OOO 기자" 등의 불필요한 텍스트 제거
    while text_list:
        if p0.search(string=text_list[-1]) is None:
            break
        del text_list[-1]
        
    content = '\n\n'.join(text_list).strip(' \t\n\r\f\v')

    # 5. 뉴스 웹사이트 이름
    website = 'Hankyung'

    # 6. 뉴스 카테고리
    category = None
    
    # 7. 비고
    note = None

    info['news_title'] = title
    info['news_first_upload_time'] = first_upload_time
    info['news_last_upload_time'] = last_upload_time
    info['author'] = author
    info['news_content'] = content
    info['news_url'] = news_url
    info['news_website'] = website
    info['news_category'] = category
    info['note'] = note

    return info

In [11]:
def crawling(website: str, news_websites_dict: dict[str, str]):
    """Crawling을 하는 함수

    Args:
        website: 웹사이트 이름, str
        news_websites_dict: 웹사이트 Dictionary, dict[str, str]

    Returns:
        pass
    """

    assert website in news_websites_dict, f'{website}은 대상 웹사이트가 아닙니다.'

    match website:
        case 'Investing':
            pass

        case 'Hankyung':
            pass

        case 'Bloomingbit':
            pass

        case 'Coinreaders':
            pass

# 전역 변수 및 환경 설정

In [12]:
# cpu 갯수
workers = os.cpu_count()
print(workers)

8


In [13]:
news_websites_dict = {
                        'Investing': 'https://kr.investing.com/news/cryptocurrency-news',
                        'Hankyung': 'https://www.hankyung.com/koreamarket/news/crypto',
                        'Bloomingbit': 'https://bloomingbit.io/feed',
                        'Coinreaders': 'https://www.coinreaders.com/'
                      }

In [14]:
# User-Agent 변경을 위한 옵션 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': user_agent}

# requests 파라미터
allow_redirects = True # 리다이렉트 허용 여부
timeout = 90 # 응답 대기 허용 시간
max_retry = 10 # HTML 문서 요청 최대 재시도 횟수

# 경로
investing_path = r'C:\Users\User\Desktop\STFO_Project\datas\news_data\Investing_Data.json'
hankyung_path = r'C:\Users\User\Desktop\STFO_Project\datas\news_data\Hankyung_Data.json'

# Main

## https://kr.investing.com/news/cryptocurrency-news

### 크롤링

In [21]:
web_page = 'https://kr.investing.com/news/cryptocurrency-news'
start = 1
get_page_cnt = 30
end = start + get_page_cnt
p0 = re.compile(pattern=r'\s+\r\n\s+')
p1 = re.compile(pattern=r'\n+')
p2 = re.compile(pattern=r'(읽기|provided|[a-z0-9]@[a-z0-9]|무단전재 및 재배포|이 글의 독자는 인베스팅프로 결제 시 쿠폰 코드|인베스팅닷컴 유저를 위한|지금 인베스팅프로)', flags=re.IGNORECASE)
p3 = re.compile(pattern=r'([가-힣])\.(\w)')
p4 = re.compile(pattern=r'\.━')
p5 = re.compile(pattern=r'\n{3,}')
# 일부 파라미터들을 고정한 investing 함수 생성
fixed_investing = partial(investing,
                            headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry,
                            p0=p0, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5)
investing_results = []

for i in tqdm(range(start, end), mininterval=1, miniters=1):
    try:
        html = request_html(url=web_page, headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry)
        soup = BeautifulSoup(html, 'html.parser')
        url_tag_list = soup.find_all('article', {"data-test": "article-item"})
        url_list = [url["href"] for url_tag in url_tag_list if ((url := url_tag.find('a')) is not None)]
    except Exception as e:
        print()
        print(f'{i}번 페이지의 HTML 문서 정보를 불러오는데 실패했습니다.')
        print(traceback.format_exc())
        web_page = f'https://kr.investing.com/news/cryptocurrency-news/{i + 1}'
        continue

    try:
        with futures.ThreadPoolExecutor(max_workers=workers) as executor:
            data = executor.map(fixed_investing, url_list)

        for idx, d in enumerate(data, start=1):
            if d is not None:
                investing_results.append(d)
            else:
                print()
                print(f'-{i}번 페이지-')
                print(f'{i}번 페이지의 {idx}번째 데이터를 가져오는 것에 실패했습니다.')
                print(f'실패한 뉴스 데이터의 URL : {url_list[idx - 1]}')

    except Exception as e:
        print()
        print(f'-{i}번 페이지-')
        print(traceback.format_exc())

    time.sleep(random.uniform(0.5, 1.25))
    web_page = f'https://kr.investing.com/news/cryptocurrency-news/{i + 1}'

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [03:07<00:00,  6.26s/it]


### JSON으로 저장

In [33]:
with open(investing_path, 'w', encoding='utf-8') as f:
    json.dump(investing_results, f, ensure_ascii=False, indent=4)

## https://www.hankyung.com/koreamarket/news/crypto

### 크롤링

In [28]:
start = 1
get_page_cnt = 200
end = start + get_page_cnt
p0 = re.compile(pattern=r'([a-z0-9]@[a-z0-9]|사진=|\b기자\b|영상촬영\s*:|한국경제TV [가-힣]{2,4}입니다)', flags=re.IGNORECASE)

# 일부 파라미터들을 고정한 hankyung 함수 생성
fixed_hankyung = partial(hankyung,
                            headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry,
                            p0=p0)
hankyung_results = []

for i in tqdm(range(start, end)):
    web_page = f'https://www.hankyung.com/koreamarket/news/crypto?page={i}'
    try:
        html = request_html(url=web_page, headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry)
        soup = BeautifulSoup(html, 'html.parser')
        url_tag_list = soup.find_all('h2', {"class": "news-tit"})
        url_list = [url["href"] for url_tag in url_tag_list if ((url := url_tag.find('a')) is not None)]
    except Exception as e:
        print()
        print(f'{i}번 페이지의 HTML 문서 정보를 불러오는데 실패했습니다.')
        print(traceback.format_exc())
        continue

    try:
        with futures.ThreadPoolExecutor(max_workers=workers) as executor:
            data = executor.map(fixed_hankyung, url_list)

        for idx, d in enumerate(data, start=1):
            if d is not None:
                hankyung_results.append(d)
            else:
                print()
                print(f'-{i}번 페이지-')
                print(f'{i}번 페이지의 {idx}번째 데이터를 가져오는 것에 실패했습니다.')
                print(f'실패한 뉴스 데이터의 URL : {url_list[idx - 1]}')

    except Exception as e:
        print()
        print(f'-{i}번 페이지-')
        print(traceback.format_exc())

    time.sleep(random.uniform(0.5, 1.25))

100%|██████████| 200/200 [11:42<00:00,  3.51s/it]


### JSON으로 저장

In [29]:
with open(hankyung_path, 'w', encoding='utf-8') as f:
    json.dump(hankyung_results, f, ensure_ascii=False, indent=4)

# 테스트

In [29]:
url = 'https://kr.investing.com/news/cryptocurrency-news/article-1301235'
p0 = re.compile(pattern=r'\s+\r\n\s+')
p1 = re.compile(pattern=r'\n+')
p2 = re.compile(pattern=r'(읽기|provided|[a-z0-9]@[a-z0-9]|무단전재 및 재배포|이 글의 독자는 인베스팅프로 결제 시 쿠폰 코드|인베스팅닷컴 유저를 위한|지금 인베스팅프로)', flags=re.IGNORECASE)
p3 = re.compile(pattern=r'([가-힣])\.(\w)')
p4 = re.compile(pattern=r'\.━')
p5 = re.compile(pattern=r'\n{3,}')
# 일부 파라미터들을 고정한 investing 함수 생성
fixed_investing = partial(investing,
                            headers=headers, allow_redirects=allow_redirects, timeout=timeout, max_retry=max_retry,
                            p0=p0, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5)

res = fixed_investing(url)
print(res['news_content'])

비트코인(BTC) 현물 상장지수펀드(ETF)가 순유입 기조를 이어갔다.

11일(현지시간) 트레이더 T 및 파사이드 인베스터에 따르면 이날 비트코인 현물 ETF에는 총 2억2310만달러가 순유입됐다.

블랙록 IBIT가 순유입을 전혀 기록하지 못했지만, 피델리티 FBTC(+1억2190만달러), 아크인베스트 ARKB(+5270만달러), 그레이스케일 GBTC(+2010만달러), 그레이스케일 BTC(+1570만달러), 비트와이즈 BITB(+1220만달러), 반에크 HODL(+290만달러) 등이 순유입을 기록했다.

반면 발키리 BRRR은 240만달러를 순유출했다.
